# MOD_SIM_P_3 Workspace

By Corey Cochran-Lepiz,
   Shawn Albertson
   
Orbital Trajectories

### First step in any good code:import your libraries!


In [2]:
from math import *
from modsim import *

### Setup your units

In [3]:
m = UNITS.meter
km = UNITS.kilometer
s = UNITS.second
kg = UNITS.kilogram
N = kg * m/(s**2)
kN = kg * m/(s**2) * 1000
G = 6.67408 * (10**-11) * m**3 * kg**-1 * s**-2


In [16]:
mars = System(x= 0 * m,
              y= 0 * m,
              radius = 3396200 * m,
              mass = .64171 * (10**24) * kg    
                )

rocket = System(x= -1 * mars.radius,
                y= 0 * m,
                dry_mass = 1433788 * kg,
                rho = 0.02 * kg/m**3, #needs function to update
                h = 0 * m, #height
                thrust = 22800000 * N,
                fuel_init = 1000000 * kg
               )


## Things to make for next time
### A function that calculates:
### Height, force of drag, gravity (with vector pointing 0,0), rho (air density), mass (as we lose propellant), thrust (stages?).

In [ ]:
def calc_mass_r(t):
    rocket.fuel_init * -.18 * t
    return rocket.dry_mass + rocket.fuel_init
    
calc_mass_rocket(30)

In [26]:
def calc_grav(height, mass_rocket):
    # G is the gravitational constant
    return G * mars.mass * mass_rocket / (height * m + mars.radius)**2


In [27]:
a = calc_mass_r(100)
calc_grav(3000, a)

<Quantity(9021100.56992121, 'kilogram * meter / second ** 2')>

In [10]:
def height_from_mars(x,y): #Assuming that mars is at the origin
    #[theta,rho] = cart2pol(x,y) #but it doens't like big numbers
    x /= m #strip the units
    y /= m
    rho = sqrt(x**2+y**2) * m #distance formula
    height = rho - (3396200*m)
    return height

def force_of_drag(v,h):
    # rho is the air density
    # v is velocity of rocket
    # c_d is drag coeffeciant of drag of rocket
    # but it's dependant on velocity
    C_d = drag(v)
    return -(1/2) * rho * v.mag * v * C_d * area

def rho(h): #math from NASA https://www.grc.nasa.gov/www/k-12/airplane/atmosmrm.html
    if h < 7000: #two zones for density
        T = -31 - 0.000998 * h #temp
        P = .699*exp(-0.00009*h) #pressure
    else:
        T = -23.4 - 0.00222*h
        P = .699*exp(-0.00009*h)
    return P/(.1921*(T+273.1)) #return density

In [ ]:
def slope_func(state,t,system):
    x, y, vx, vy = state
    unpack(system)
    
    h = height_from_mars(x,y)
    a_grav = calc_grav(h)
    
    v = Vector(vx,vy)
    
    f_drag = force_of_drag(v.mag,h)
    a_drag = d_drag/mass
    
    a = a_grav + a_drag + a_thrust
    
    return v.x, v.y, a.x, a.y